In [1]:
from numbers import Integral

In [2]:
class Person:
    @property
    def age(self):
        return getattr(self, "_age", None)
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError("age: must be an integer.")
        if value < 0:
            raise ValueError("age: must be a non-negative integer.")
        self._age = value

without decorators this is equivalent to:

In [ ]:
class Person:
    def age(self):
        return getattr(self, "_age", None)
    age = property(age)
    
    
    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError("age: must be an integer.")
        if value < 0:
            raise ValueError("age: must be a non-negative integer.")
        self._age = value
        
    age = age.setter(set_age)

In [3]:
p = Person()

In [4]:
p.age

In [5]:
p.age = -10

ValueError: age: must be a non-negative integer.

In [6]:
p.age = 10

In [7]:
p.__dict__ #property age is not in the dict

{'_age': 10}

let's rewrite this class without decorators

In [8]:
class Person:
    def get_age(self):
        return getattr(self, "_age", None)

    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError("age: must be an integer.")
        if value < 0:
            raise ValueError("age: must be a non-negative integer.")
        self._age = value
        
    age = property(fget=get_age, fset=set_age)

In [12]:
prop = Person.age
hasattr(prop, "__get__"), hasattr(prop, "__set__"), hasattr(prop, "__delete__")

(True, True, True)

In [13]:
p = Person()
p.age = 45
p.age

45

In [14]:
class TimeUTC:
    @property
    def current_time(self):
        return "current time"

In [15]:
hasattr(TimeUTC.current_time, "__get__"), hasattr(TimeUTC.current_time, "__set__"), hasattr(TimeUTC.current_time, "__delete__")

(True, True, True)

In [16]:
t = TimeUTC()
t.current_time

'current time'

In [17]:
t.current_time = 20 #because we didn't define fset method

AttributeError: can't set attribute

#### since properties are data descriptors, instance dict will not shadow the descriptor name

In [18]:
p = Person()
p.__dict__

{}

In [19]:
p.age = 10
p.__dict__

{'_age': 10}

In [20]:
p.__dict__['age'] = 200
p.__dict__

{'_age': 10, 'age': 200}

In [21]:
p.age

10

#### let's create our own class which will create properties. 
it will be a rough approximation

In [9]:
class MakeProperty:
    def __init__(self, fget=None, fset=None):
        self.fget=fget
        self.fset=fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __get__(self, instance, owner_class): # instance of the descriptor, 
        # instance of the object, that we are calling the descriptor on,
        # and owner class
        print("__get__ called")
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f"{self.prop_name} is not readable.")
        return self.fget(instance)
            
    def __set__(self, instance, value):
        print("__set__ called") 
        if self.fset is None:
            raise AttributeError(f"{self.prop_name} is not writable.")
        self.fset(instance, value)    
            
            
            

#### let's create properties using our MakeProperty class

In [10]:
class Person:
    def get_name(self):
        print("get_name called")
        return getattr(self, "_name", None)

    def set_name(self, value):
        print("set_name called")
        self._name = value
        
    name = MakeProperty(fget=get_name, fset=set_name)

In [11]:
p = Person()
p.name = "Zena"

__set__ called
set_name called


In [12]:
p.name

__get__ called
get_name called


'Zena'

In [13]:
p.__dict__

{'_name': 'Zena'}

and instance dict does not shadow a data descriptor

In [14]:
p.__dict__['name'] = "Sew"

In [15]:
p.name

__get__ called
get_name called


'Zena'

#### let's do decorator approach as well

we need two decorators

In [16]:
class Person:
    @MakeProperty
    def age(self):
        return 100 

In [17]:
p = Person()
Person.age

__get__ called


In [18]:
p.age

__get__ called


100

#### we want to use setter decorator as well

In [19]:
class MakeProperty:
    def __init__(self, fget=None, fset=None):
        self.fget=fget
        self.fset=fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __get__(self, instance, owner_class): # instance of the descriptor, 
        # instance of the object, that we are calling the descriptor on,
        # and owner class
        print("__get__ called")
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f"{self.prop_name} is not readable.")
        return self.fget(instance)
            
    def __set__(self, instance, value):
        print("__set__ called") 
        if self.fset is None:
            raise AttributeError(f"{self.prop_name} is not writable.")
        self.fset(instance, value)    
        
    def setter(self, fset):
        # with properties it actually creates a new instance of the property
        # and then populates fget and fset
        self.fset=fset
        return self
    
    
            

In [20]:
class Person:
    @MakeProperty
    def age(self):
        return getattr(self, "_age", None) 
    
    @age.setter
    def age(self, value):
        self._age = value
    

In [21]:
p = Person()
p.age = 100

__set__ called


In [22]:
p.age

__get__ called


100